In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import PIL.Image as Image
import os
import glob
from pathlib import Path
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt
import tqdm.notebook as notebook
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score
import io
from google.cloud import storage

ModuleNotFoundError: No module named 'google'

In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 500.8 kB/s eta 0:00:000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 22.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 16.8 MB/s eta 0:00:0000:0100:01


In [5]:
os.getcwd()

'/home/jupyter'

In [6]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

In [9]:
# Inititiation Block
Z_START = 25
Z_DIM = 10
BUFFER = 30
BATCH_SIZE = 128

In [7]:
client = storage.Client()
bucket = client.get_bucket('dlproject-7643')
def get_img_bucket(text):
    blob = bucket.blob(text)
    img_bytes = io.BytesIO(blob.download_as_bytes())
    img = Image.open(img_bytes)
    return img


In [3]:
# gold = np.array(Image.open("/kaggle/input/vesuvius-challenge-ink-detection/train/3/inklabels.png"))
# mask=np.array(Image.open("/kaggle/input/vesuvius-challenge-ink-detection/train/3/mask.png"))
# gold2 = np.array(Image.open("/kaggle/input/vesuvius-challenge-ink-detection/train/1/inklabels.png"))

In [4]:
#visualizing output and gold label
# fig, (ax1, ax2) = plt.subplots(1, 2)
# ax1.imshow(gold2, cmap='gray')
# ax2.imshow(gold, cmap='gray')
# plt.show()

In [12]:
def load_img(x):
    PREFIX = "vesuvius-challenge-ink-detection/train/"+x+"/surface_volume/"
    # Use glob.glob to get a list of all the file paths
    file_paths = [PREFIX + "{:02d}.tif".format(i) for i in range(0, Z_DIM)]

    # Load the images into a list of numpy arrays
    images = [np.array(get_img_bucket(filename), dtype=np.float32)/65535.0 for filename in notebook.tqdm(file_paths)]
    data = np.array(images)
    gold = np.array(get_img_bucket("vesuvius-challenge-ink-detection/train/"+x+"/inklabels.png"))
    mask=np.array(get_img_bucket("vesuvius-challenge-ink-detection/train/"+x+"/mask.png"))
    return data,gold,mask


def pad_array(x,Buffer=BUFFER):
    x=  np.pad(x, ((0, 0), (Buffer, Buffer), (Buffer, Buffer)), mode='constant', constant_values=0)
    return x

def get_pixels(mask,rect = [2500,3500,1500,2500]):
    inside_rect = np.zeros(mask.shape, dtype=bool)
    inside_rect[rect[0]:rect[1], rect[2]:rect[3]] = True

    outside_rect = np.ones(mask.shape, dtype=bool)
    outside_rect[rect[0]:rect[1], rect[2]:rect[3]] = True
    
    inside_rect = torch.from_numpy(np.pad(inside_rect,((BUFFER, BUFFER), (BUFFER, BUFFER)), mode='constant', constant_values=0)).float()
    outside_rect = torch.from_numpy(np.pad(outside_rect,((BUFFER, BUFFER), (BUFFER, BUFFER)), mode='constant', constant_values=0)).float()
    val_pixels = torch.argwhere(inside_rect)
    train_pixels = torch.argwhere(outside_rect)
    return train_pixels,val_pixels

def pad_datasets(data,mask,gold):
    mask = torch.from_numpy(np.pad(mask,((BUFFER, BUFFER), (BUFFER, BUFFER)), mode='constant', constant_values=0)).float()
    gold = torch.from_numpy(np.pad(gold,((BUFFER, BUFFER), (BUFFER, BUFFER)), mode='constant', constant_values=0)).float()
    data = pad_array(data)
    data = torch.stack([torch.from_numpy(image) for image in data], dim=0)
    return data,mask,gold


In [14]:
input_data ={}
gold_labels ={}
t_pixels = []
v_pixels = []

for i in [1,3]:
    data,gold,mask = load_img(str(i))
    train_pixels,val_pixels = get_pixels(mask)
    data,mask,gold = pad_datasets(data,mask,gold)
    t_pixels.append(torch.cat((train_pixels,i*torch.ones((train_pixels.shape[0],1))),dim=1).int())
    v_pixels.append(torch.cat((val_pixels,i*torch.ones((val_pixels.shape[0],1))),dim=1).int())
    gold_labels[i] = gold
    input_data[i] = data
    
train_pixels = torch.cat(t_pixels,dim=0)
val_pixels = torch.cat(v_pixels,dim=0)

del t_pixels
del v_pixels

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 1627951760 bytes. Error code 12 (Cannot allocate memory)

In [8]:
class SubvolumeDataset(Dataset):
    def __init__(self, image_stack, label, pixels):
        self.image_stack = image_stack
        self.label = label
        self.pixels = pixels

    def __len__(self):
        return len(self.pixels)
    
    def __getitem__(self, index):
        y,x,key = self.pixels[index]
        
        try:
            subvolume = self.image_stack[int(key.item())][:, y-BUFFER:y+BUFFER+1, x-BUFFER:x+BUFFER+1].view(1, Z_DIM, BUFFER*2+1, BUFFER*2+1)
        except:
            print(self.image_stack[key.item()][:, y-BUFFER:y+BUFFER+1, x-BUFFER:x+BUFFER+1].shape)
            print(x,y,BUFFER)
            subvolume = self.image_stack[key.item()][:, y-BUFFER:y+BUFFER+1, x-BUFFER:x+BUFFER+1].view(1, Z_DIM, BUFFER*2+1, BUFFER*2+1)
        
        inklabel = self.label[key.item()][y, x].view(1)
        
        return subvolume, inklabel

In [9]:
train_dataset = SubvolumeDataset(input_data,gold_labels,train_pixels)
train_loader =  DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataset = SubvolumeDataset(input_data,gold_labels,val_pixels)
val_loader =  DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [10]:
class InkDetection(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv3d(in_channels = 1,out_channels = 16, kernel_size = 3 , padding = 1)
        self.act1 = nn.LeakyReLU()
        
        self.BN1 = nn.BatchNorm3d(16)
        self.conv2 =  nn.Conv3d(in_channels = 16,  out_channels =32 ,kernel_size = 3, padding =1)
        self.BN2 = nn.BatchNorm3d(32)
        self.conv3 =  nn.Conv3d(in_channels = 32,  out_channels =64 ,kernel_size = 3, padding =1)
        self.BN3 = nn.BatchNorm3d(64)
        
        self.pool = nn.MaxPool3d(2,2)
        
        self.ffn = nn.Sequential(nn.LazyLinear(128),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(128,128),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(128,2)
        )
        self.final = nn.Sigmoid()
        
    def forward(self,X):
        X= self.conv1(X)
        X= self.act1 (X)
        X=self.BN1(X)
        X=self.pool(X)
        X= self.conv2(X)
        X= self.act1 (X)
        X=self.BN2(X)
        X=self.pool(X)
        X= self.conv3(X)
        X= self.act1 (X)
        X=self.BN3(X)
        X=self.pool(X)

        X=X.flatten(start_dim=1)

        #linear layers
        X=self.ffn(X)
        X =self.final(X)


        return X   
    
    


In [ ]:
# #setting up the pretrained model
# model = InkDetection().to(DEVICE)
# # Loading the trained model weights
# model.load_state_dict(torch.load('/kaggle/input/model-v2/model_PB.pt'))


In [ ]:
print("Training...")

model = InkDetection().to(DEVICE)
model.zero_grad()
LEARNING_RATE = 0.0001
TRAINING_STEPS = 30000
class_weights = torch.Tensor([0.18,0.82]).to(DEVICE)
f1_running= 0

criterion = nn.BCELoss(weight=class_weights)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
i=0



num_epochs = 5

progress_bar_epoch = notebook.tqdm(range(num_epochs),desc = "Epoch Number",leave =True)



for epoch in progress_bar_epoch:
    model.train()
    f1_running= 0
    train_loss = 0
    accuracy = 0
    progress_bar = notebook.tqdm(train_loader,desc = "Batch Number",leave =True)
    #Training loop
    for i,batch in enumerate(progress_bar):
        optimizer.zero_grad()
        X,y =batch

        y_onehot = torch.nn.functional.one_hot(y.squeeze().long(),num_classes =2).float().to(DEVICE)
        # print(y_onehot.shape)


        output = model(X.to(DEVICE))

        if i>TRAINING_STEPS:
            break
        loss = criterion(output, y_onehot)
        loss.backward()
        optimizer.step()
        
        train_loss = loss.clone().detach().item()

        predicted=torch.argmax(output.clone().detach(),dim=1).unsqueeze(1)
        # predicted=(predicted>0.4).type(torch.int)

        epsilon = 1e-5


        true_positives = torch.sum(torch.mul(predicted ,y)).float()
        true_negatives = torch.sum(torch.mul((1-predicted ),(1-y))).float()
        false_positives = torch.sum(torch.mul(predicted,(1-y))).float()
        false_negatives = torch.sum(torch.mul((1-predicted), y)).float()
        positives = true_positives + false_positives
        negatives = true_negatives + false_negatives
        accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives + epsilon)
        precision = true_positives / (true_positives + false_positives + epsilon)
        
        recall = true_positives / (true_positives + false_negatives + epsilon)
        
        f1 = (1.25*precision*recall/(0.25*precision+recall+epsilon)).item()
        f1_running += f1
        if i%100 == 0:

            progress_bar.set_postfix({"Running Loss":train_loss,"Mean F-1 ":f1_running/(i+1) ,"Batch_F1 ": f1,"Accuracy ": accuracy.item() , "Total Positives in Batch ":y.sum().item(),"True Positives ":true_positives.item(), "Predicted_sum":predicted.sum().item()} )
            
    #Eval loop
    print("Evaluation after epoch no",i)
    model.eval()
    f1_running_eval= 0
    tp_running=0
    tn_running=0
    fp_running=0
    fn_running=0
#     criterion = nn.BCELoss(weight=class_weights)
    # out_img = torch.zeros_like(gold).float()
    progress_bar_val = notebook.tqdm(val_loader,desc = "Batch Number",leave =True)
    with torch.no_grad():
        for k,batch in enumerate(progress_bar_val):
            X,y =batch
            output = model.forward(X.to(DEVICE))

            # for l,value in enumerate(output):
            #     out_img[tuple(val_pixels[k*BATCH_SIZE+l])] = torch.argmax(value)

            y_onehot = torch.nn.functional.one_hot(y.squeeze().long(),num_classes =2).float().to(DEVICE)
            loss_eval = criterion(output.to(DEVICE), y_onehot.to(DEVICE))
            eval_loss = loss_eval.clone().detach().item()

            predicted=output.clone().detach()
            predicted=torch.argmax(output.clone().detach(),dim=1).unsqueeze(1)
    #         predicted=(predicted>0.4).type(torch.int)

            epsilon = 1e-5

            true_positives = torch.sum(torch.mul(predicted ,y)).float()
            true_negatives = torch.sum(torch.mul((1-predicted ),(1-y))).float()
            false_positives = torch.sum(torch.mul(predicted,(1-y))).float()
            false_negatives = torch.sum(torch.mul((1-predicted), y)).float()
            positives = true_positives + false_positives
            negatives = true_negatives + false_negatives
            accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives + epsilon)
            precision = true_positives / (true_positives + false_positives + epsilon)

            recall = true_positives / (true_positives + false_negatives + epsilon)
            f1_eval = (1.25*precision*recall/(0.25*precision+recall+epsilon)).item()
            tp_running+=true_positives
            tn_running+=true_negatives
            fp_running+=false_positives
            fn_running+=false_negatives
            f1_running_eval = (1.25*tp_running/(1.25*tp_running+0.25*fn_running+fp_running+epsilon))
            if i%100 == 0:
                progress_bar_val.set_postfix({"Running Loss":eval_loss,"Batch F-1 ":f1_eval ,"Accuracy ": accuracy.item() , "Total Positives in Batch ":y.sum().item(),"True Positives ":true_positives.item(), "Predicted_sum":predicted.sum().item()} )
                print("Evaluation after epoch no",i,' : ',f1_running_eval.item())
        
    


    progress_bar_epoch.set_postfix({"Training Loss": train_loss , "Train F-1": f1_running/(i+1)})


Training...


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Epoch Number:   0%|          | 0/5 [00:00<?, ?it/s]

Batch Number:   0%|          | 0/7813 [00:00<?, ?it/s]

Batch Number:   0%|          | 0/311906 [00:00<?, ?it/s]

In [ ]:
# print("Training...")

# # model = InkDetection().to(DEVICE)
# model.zero_grad()
# LEARNING_RATE = 0.001
# TRAINING_STEPS = 30000
# class_weights = torch.Tensor([0.18,0.82]).to(DEVICE)
# f1_running= 0
# model.train()
# criterion = nn.BCELoss(weight=class_weights)

# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
# i=0



# num_epochs = 5

# progress_bar_epoch = notebook.tqdm(range(num_epochs),desc = "Epoch Number",leave =True)


# for epoch in progress_bar_epoch:
#     f1_running= 0
#     train_loss = 0
#     accuracy = 0
#     progress_bar = notebook.tqdm(train_loader,desc = "Batch Number",leave =True)
#     for i,batch in enumerate(progress_bar):
#         optimizer.zero_grad()
#         X,y =batch

#         y_onehot = torch.nn.functional.one_hot(y.squeeze().long(),num_classes =2).float().to(DEVICE)
#         # print(y_onehot.shape)


#         output = model(X.to(DEVICE))

#         if i>TRAINING_STEPS:
#             break
#         loss = criterion(output, y_onehot)
#         loss.backward()
#         optimizer.step()
        
#         train_loss = loss.clone().detach().item()

#         predicted=torch.argmax(output.clone().detach(),dim=1).unsqueeze(1)
#         # predicted=(predicted>0.4).type(torch.int)

#         epsilon = 1e-5


#         true_positives = torch.sum(torch.mul(predicted ,y)).float()
#         true_negatives = torch.sum(torch.mul((1-predicted ),(1-y))).float()
#         false_positives = torch.sum(torch.mul(predicted,(1-y))).float()
#         false_negatives = torch.sum(torch.mul((1-predicted), y)).float()
#         positives = true_positives + false_positives
#         negatives = true_negatives + false_negatives
#         accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives + epsilon)
#         precision = true_positives / (true_positives + false_positives + epsilon)
        
#         recall = true_positives / (true_positives + false_negatives + epsilon)
        
#         f1 = (1.25*precision*recall/(0.25*precision+recall+epsilon)).item()
#         f1_running += f1
#         if i%100 == 0:

#             progress_bar.set_postfix({"Running Loss":train_loss,"Mean F-1 ":f1_running/(i+1) ,"Batch_F1 ": f1,"Accuracy ": accuracy.item() , "Total Positives in Batch ":y.sum().item(),"True Positives ":true_positives.item(), "Predicted_sum":predicted.sum().item()} )


#     progress_bar_epoch.set_postfix({"Training Loss": train_loss , "Train F-1": f1_running/(i+1)})


In [ ]:
torch.save(model.state_dict(), 'model_seq_1_and_3.pt')

In [ ]:
val_dataset = SubvolumeDataset(data,gold,val_pixels)
val_loader =  DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model = InkDetection().to(DEVICE)
model.load_state_dict(torch.load('/kaggle/input/model-v2/model_PB.pt'))

In [ ]:
model

In [ ]:
model.eval()
progress_bar = notebook.tqdm(val_loader,desc = "Batch Number",leave =True)
class_weights = torch.Tensor([0.18,0.82]).to(DEVICE)
f1_running= 0
tp_running=0
tn_running=0
fp_running=0
fn_running=0
criterion = nn.BCELoss(weight=class_weights)
out_img = torch.zeros_like(gold).float()
with torch.no_grad():
    for i,batch in enumerate(progress_bar):
        X,y =batch
        output = model.forward(X.to(DEVICE))
    
        for j,value in enumerate(output):
            out_img[tuple(val_pixels[i*BATCH_SIZE+j])] = torch.argmax(value)
            
        y_onehot = torch.nn.functional.one_hot(y.squeeze().long(),num_classes =2).float().to(DEVICE)
        loss = criterion(output.to(DEVICE), y_onehot.to(DEVICE))
        train_loss = loss.clone().detach().item()

        predicted=output.clone().detach()
        predicted=torch.argmax(output.clone().detach(),dim=1).unsqueeze(1)
#         predicted=(predicted>0.4).type(torch.int)

        epsilon = 1e-5

        true_positives = torch.sum(torch.mul(predicted ,y)).float()
        true_negatives = torch.sum(torch.mul((1-predicted ),(1-y))).float()
        false_positives = torch.sum(torch.mul(predicted,(1-y))).float()
        false_negatives = torch.sum(torch.mul((1-predicted), y)).float()
        positives = true_positives + false_positives
        negatives = true_negatives + false_negatives
        accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives + epsilon)
        precision = true_positives / (true_positives + false_positives + epsilon)

        recall = true_positives / (true_positives + false_negatives + epsilon)
        f1 = (1.25*precision*recall/(0.25*precision+recall+epsilon)).item()
        tp_running+=true_positives
        tn_running+=true_negatives
        fp_running+=false_positives
        fn_running+=false_negatives
        f1_running = (1.25*tp_running/(1.25*tp_running+0.25*fn_running+fp_running+epsilon))
        if i%100 == 0:
            progress_bar.set_postfix({"Running Loss":train_loss,"Batch F-1 ":f1 ,"Accuracy ": accuracy.item() , "Total Positives in Batch ":y.sum().item(),"True Positives ":true_positives.item(), "Predicted_sum":predicted.sum().item()} )
            print(f1_running.item())


In [ ]:
#visualizing output and gold label
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(out_img.cpu()[rect[0]:rect[1],rect[2]:rect[3]], cmap='gray')
ax2.imshow(gold.cpu()[rect[0]:rect[1],rect[2]:rect[3]], cmap='gray')
plt.show()

In [ ]:
print("Training...")

# model = InkDetection().to(DEVICE)
model.zero_grad()
LEARNING_RATE = 0.0001
TRAINING_STEPS = 5000
class_weights = torch.Tensor([0.18,0.82]).to(DEVICE)
f1_running= 0

criterion = nn.BCELoss(weight=class_weights)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
i=0



num_epochs = 5

progress_bar_epoch = notebook.tqdm(range(num_epochs),desc = "Epoch Number",leave =True)


for epoch in progress_bar_epoch:
    model.train()
    f1_running= 0
    train_loss = 0
    accuracy = 0
    progress_bar = notebook.tqdm(train_loader,desc = "Batch Number",leave =True)
    #Training loop
    for i,batch in enumerate(progress_bar):
        optimizer.zero_grad()
        X,y =batch

        y_onehot = torch.nn.functional.one_hot(y.squeeze().long(),num_classes =2).float().to(DEVICE)
        # print(y_onehot.shape)


        output = model(X.to(DEVICE))

        if i>TRAINING_STEPS:
            break
        loss = criterion(output, y_onehot)
        loss.backward()
        optimizer.step()
        
        train_loss = loss.clone().detach().item()

        predicted=torch.argmax(output.clone().detach(),dim=1).unsqueeze(1)
        # predicted=(predicted>0.4).type(torch.int)

        epsilon = 1e-5


        true_positives = torch.sum(torch.mul(predicted ,y)).float()
        true_negatives = torch.sum(torch.mul((1-predicted ),(1-y))).float()
        false_positives = torch.sum(torch.mul(predicted,(1-y))).float()
        false_negatives = torch.sum(torch.mul((1-predicted), y)).float()
        positives = true_positives + false_positives
        negatives = true_negatives + false_negatives
        accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives + epsilon)
        precision = true_positives / (true_positives + false_positives + epsilon)
        
        recall = true_positives / (true_positives + false_negatives + epsilon)
        
        f1 = (1.25*precision*recall/(0.25*precision+recall+epsilon)).item()
        f1_running += f1
        if i%100 == 0:

            progress_bar.set_postfix({"Running Loss":train_loss,"Mean F-1 ":f1_running/(i+1) ,"Batch_F1 ": f1,"Accuracy ": accuracy.item() , "Total Positives in Batch ":y.sum().item(),"True Positives ":true_positives.item(), "Predicted_sum":predicted.sum().item()} )
            
    #Eval loop
    print("Evaluation after epoch no",i)
    model.eval()
    f1_running_eval= 0
    tp_running=0
    tn_running=0
    fp_running=0
    fn_running=0
#     criterion = nn.BCELoss(weight=class_weights)
    out_img = torch.zeros_like(gold).float()
    with torch.no_grad():
        for k,batch in enumerate(progress_bar_val):
            X,y =batch
            output = model.forward(X.to(DEVICE))

            for l,value in enumerate(output):
                out_img[tuple(val_pixels[k*BATCH_SIZE+l])] = torch.argmax(value)

            y_onehot = torch.nn.functional.one_hot(y.squeeze().long(),num_classes =2).float().to(DEVICE)
            loss_eval = criterion(output.to(DEVICE), y_onehot.to(DEVICE))
            eval_loss = loss_eval.clone().detach().item()

            predicted=output.clone().detach()
            predicted=torch.argmax(output.clone().detach(),dim=1).unsqueeze(1)
    #         predicted=(predicted>0.4).type(torch.int)

            epsilon = 1e-5

            true_positives = torch.sum(torch.mul(predicted ,y)).float()
            true_negatives = torch.sum(torch.mul((1-predicted ),(1-y))).float()
            false_positives = torch.sum(torch.mul(predicted,(1-y))).float()
            false_negatives = torch.sum(torch.mul((1-predicted), y)).float()
            positives = true_positives + false_positives
            negatives = true_negatives + false_negatives
            accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives + epsilon)
            precision = true_positives / (true_positives + false_positives + epsilon)

            recall = true_positives / (true_positives + false_negatives + epsilon)
            f1_eval = (1.25*precision*recall/(0.25*precision+recall+epsilon)).item()
            tp_running+=true_positives
            tn_running+=true_negatives
            fp_running+=false_positives
            fn_running+=false_negatives
            f1_running_eval = (1.25*tp_running/(1.25*tp_running+0.25*fn_running+fp_running+epsilon))
            if i%100 == 0:
                progress_bar_val.set_postfix({"Running Loss":eval_loss,"Batch F-1 ":f1_eval ,"Accuracy ": accuracy.item() , "Total Positives in Batch ":y.sum().item(),"True Positives ":true_positives.item(), "Predicted_sum":predicted.sum().item()} )
                print("Evaluation after epoch no",i,' : ',f1_running_eval.item())
        
    


    progress_bar_epoch.set_postfix({"Training Loss": train_loss , "Train F-1": f1_running/(i+1)})


In [ ]:

progress_bar_val = notebook.tqdm(val_loader,desc = "Batch Number",leave =True)
class_weights = torch.Tensor([0.18,0.82]).to(DEVICE)
model.eval()
f1_running= 0
tp_running=0
tn_running=0
fp_running=0
fn_running=0
criterion = nn.BCELoss(weight=class_weights)
out_img = torch.zeros_like(gold).float()
with torch.no_grad():
    for k,batch in enumerate(progress_bar_val):
        X,y =batch
        output = model.forward(X.to(DEVICE))
    
        for l,value in enumerate(output):
            out_img[tuple(val_pixels[k*BATCH_SIZE+l])] = torch.argmax(value)
            
        y_onehot = torch.nn.functional.one_hot(y.squeeze().long(),num_classes =2).float().to(DEVICE)
        loss_eval = criterion(output.to(DEVICE), y_onehot.to(DEVICE))
        train_loss = loss_eval.clone().detach().item()

        predicted=output.clone().detach()
        predicted=torch.argmax(output.clone().detach(),dim=1).unsqueeze(1)
#         predicted=(predicted>0.4).type(torch.int)

        epsilon = 1e-5

        true_positives = torch.sum(torch.mul(predicted ,y)).float()
        true_negatives = torch.sum(torch.mul((1-predicted ),(1-y))).float()
        false_positives = torch.sum(torch.mul(predicted,(1-y))).float()
        false_negatives = torch.sum(torch.mul((1-predicted), y)).float()
        positives = true_positives + false_positives
        negatives = true_negatives + false_negatives
        accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives + epsilon)
        precision = true_positives / (true_positives + false_positives + epsilon)

        recall = true_positives / (true_positives + false_negatives + epsilon)
        f1 = (1.25*precision*recall/(0.25*precision+recall+epsilon)).item()
        tp_running+=true_positives
        tn_running+=true_negatives
        fp_running+=false_positives
        fn_running+=false_negatives
        f1_running = (1.25*tp_running/(1.25*tp_running+0.25*fn_running+fp_running+epsilon))
        if i%100 == 0:
            progress_bar_val.set_postfix({"Running Loss":train_loss,"Batch F-1 ":f1 ,"Accuracy ": accuracy.item() , "Total Positives in Batch ":y.sum().item(),"True Positives ":true_positives.item(), "Predicted_sum":predicted.sum().item()} )
            print(f1_running.item())


In [ ]:
# rect = [1500,2500,3500,4500]
# rect=[4500,5500,2000,3000]
# rect=[7200,8300,1500,2500]
# rect = [2500,3500,1500,2500]
# inside_rect = np.zeros(mask.shape, dtype=bool)
# inside_rect[rect[0]:rect[1], rect[2]:rect[3]] = True

# outside_rect = np.ones(mask.shape, dtype=bool)
# outside_rect[rect[0]:rect[1], rect[2]:rect[3]] = True
# data = pad_array(data)
# mask = torch.from_numpy(np.pad(mask,((BUFFER, BUFFER), (BUFFER, BUFFER)), mode='constant', constant_values=0)).float().to(DEVICE)
# gold = torch.from_numpy(np.pad(gold,((BUFFER, BUFFER), (BUFFER, BUFFER)), mode='constant', constant_values=0)).float().to(DEVICE)
# data = torch.stack([torch.from_numpy(image) for image in data], dim=0).to(DEVICE)

# inside_rect = torch.from_numpy(np.pad(inside_rect,((BUFFER, BUFFER), (BUFFER, BUFFER)), mode='constant', constant_values=0)).float().to(DEVICE)
# outside_rect = torch.from_numpy(np.pad(outside_rect,((BUFFER, BUFFER), (BUFFER, BUFFER)), mode='constant', constant_values=0)).float().to(DEVICE)
# val_pixels = torch.argwhere(inside_rect)
# train_pixels = torch.argwhere(outside_rect)

### Balancing Datasets
# train_pixels, val_pixels = train_test_split(pixels, test_size=0.2)
# train_dataset = SubvolumeDataset(data,gold,train_pixels)
# train_loader =  DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_dataset = SubvolumeDataset(data,gold,val_pixels)
# val_loader =  DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)